### pylegoclassifier workbook
### magnus wood, december 2020, bsyse 530 semester project
The below code block will be used in the 'pylegoclassifer.py' module. It will be used in the matlab integration, where images obtained by Eric will use functions from this code to do lego color classification.

This jupyter notebook exists solely for developing it. I should probably share it too.

### pylegoclassifier.py functionality
### The code needs to do this:

1. Take an image file in and ensure it is in the right format.
2. Perform background segmentation using ImageSegmentation.
3. Data extraction:
    a. 
    b. 
4. Pass the dataframe to the 

In [5]:
%%writefile pylegoclassifier.py

# import the needed packages
import pickle
from sklearn import preprocessing
import time
from os import listdir
from os.path import isfile, join
from random import randint, uniform
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
from scipy import ndimage
from skimage import morphology
from skimage import exposure
import os
from math import pi
from math import isnan
import pandas as pd
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from skimage.filters import sobel


# set random seed
np.random.seed(26)

# the NaiveBayes classifier I wrote for assignment 6 in BSYSE_530, modified a little for this purpose
class NaiveBayes:
    # P(c|x) = P(x|c) * P(c) / P(x)
    # P(x|x) is the posterior probability
    # P(x|c) is the likelihood
    # P(c) is the class prior probability, or the prob of c occuring indpendently. 
    # P(x) is the predictor prior probability, or the prob of x occuring independently
    
    def fit(self, features, target):
        # define class variables
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        # calculate statistics for all those features
        self.calc_statistics(features, target)
        
        # prior is the random chance of drawing a particular class based on its proportion in the dataset
        self.prior = self.calc_prior(features, target)
        
              
    def get_predictions(self, input_vector):
        predictions = []
        
        for i in range(len(input_vector)):
            result = self.calc_posterior((input_vector.iloc[i,:]))
            predictions.append(result)
        return predictions
     

    def predict(self, observation):
        #call the calc_posterior function on the observation
        pred_class = self.calc_posterior(observation)
        return pred_class
        
        
    def calc_statistics(self, features, target):
        # calculate mean, variance for each column and convert to numpy array
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
        return self.mean, self.var
    
    
    def calc_prior(self, features, target):
        # this is the probability of picking one of a class at random from the dataset
        self.prior = (features.groupby(target).apply(lambda x: len(x)/self.rows).to_numpy())
        return self.prior
    
    
    def calc_posterior(self, x):
        # this is the probability, post evidence
        # x is a numpy array
        # x is feature vector for one observation 
                
        # make a list that we will add each classes posterior prob to
        posteriors = []
        
        # iterate through the classes
        for i in range(0, self.count):
            # for each class look at the prior probability for the class
            prior = self.prior[i]
            
            # calculate the conditional probability for the 
            conditional = np.sum(self.gaussian_density(i, x))
            posterior = prior + conditional
            #  print(f"i = {i}, prior = {prior}, conditional = {conditional}, posterior = {posterior}")
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]
        
        
    def gaussian_density(self, class_idx, x):
        # calc probability from gaussian denssityy fucntion (normal dist)
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        # this part sucked and I had a typo that cost me hours
        numerator = np.exp(-((x-mean)**2 / (2 * var)))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
        
    
    def pdf(self, x, mean, stdev):
        # calculate probability density function
        exponent = np.exp(-((x-mean)**2 / (2*stdev**2)))
        return exponent * (1/(np.sqrt(2*np.pi)*stdev))

        
    def get_accuracy(self, test, predictions):
        correct = 0
        for i in range(len(test)):
            if test.iloc[i] == predictions[i]:
                correct += 1
        return (correct / float(len(test)))
    


# TODO: read these and see how it works        
# https://www.mathworks.com/help/matlab/matlab_external/matlab-arrays-as-python-variables.html        
# https://www.mathworks.com/help/matlab/matlab_external/passing-data-to-python.html        
            
# this exists only for my testing purposes
class MatlabSurrogate():
    def __init__(self):
        self.state_of_mind = "Badass."
        
        
    def acquire_kinect_image(self, filename):
        # give this function a filename, and it will load that image with opencv
        # this will be a BGR format, because that is how opencv rolls
        kinect_image = cv.imread(filename)
        print(f"kinect has acquired the image with shape = {kinect_image.shape}")
        return kinect_image
    
    
    # function to display images resized, using opencv
    def imshow(self, image, imdiv = 4):
        imdiv = int(imdiv)
        w, h = int(image.shape[1]/imdiv), int(image.shape[0]/imdiv)
        cv.namedWindow("output", cv.WINDOW_NORMAL)
        cv.resizeWindow("output", (w, h))
        cv.imshow("output", image)
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    
# I should probably have one image processing class that takes in a single image and then spits out a dataframe that could be used for prediction
# replaces ImageSegmenter
class ImageProcess():
    def __init__(self):
        print("image processor activated! use 'process_image_to_df()' to get back a pandas df")
        self.black_lower = (0, 0, 0)
        self.black_upper = (179, 255, 30)
        self.hsv_lower = (0, 0, 0)
        self.hsv_upper = (179, 255, 90)
#         self.black_lower = (0, 0, 203)
#         self.black_upper = (43, 255, 255)
#         self.hsv_lower = (0, 0, 70)
#         self.hsv_upper = (179, 34, 255)
# NOT mask for lego_imgs[14]
# hsv_lower = (0,0,0)
# hsv_upper = (179,234,77)

    

    def dummy_method(self, a):
        if type(a) is np.ndarray:
            result = "object is a numpy.ndarray, this is perfect. Is the image RGB order or BGR?"
            return result
        else:
            result = "object is a " + str(type(a)) + "and I'm gonna have a hard time with that"
            return result
      
    
        
    def bg_segmentation(self, image, mode="hsv", show_img=False):
                
            # create an hsv mask for red colors
            hsv_mask = cv.inRange(cv.cvtColor(image, cv.COLOR_BGR2HSV), 
                                 self.hsv_lower,
                                 self.hsv_upper).astype(np.uint8)
             
            # use this as a NOT mask
            hsv_mask = np.where(hsv_mask > 1, 0, 1).astype(np.uint8)
  
            hsv_mask = ndimage.gaussian_filter(hsv_mask, sigma=1)

            # erode the mask
            hsv_mask = morphology.erosion(hsv_mask, morphology.disk(3))

#             # median filter to despeckle
#             hsv_mask = ndimage.median_filter(hsv_mask, size=(3, 3)).astype(np.uint8)

            # binary dilation 
            hsv_mask = morphology.binary_dilation(hsv_mask, np.ones((20, 20))).astype(np.uint8)

            # fill the holes
            hsv_mask = ndimage.binary_fill_holes(hsv_mask).astype(np.uint8)

            # erode the mask
            hsv_mask = morphology.erosion(hsv_mask, morphology.disk(5))
            
            # TODO: remove this it is for testing purposes to show the segmentation
            if (show_img == True):                
                m = MatlabSurrogate()
                m.imshow(cv.bitwise_and(image, image, mask=hsv_mask).astype(np.uint8))
            
            # apply the mask and return the result        
            return cv.bitwise_and(image, image, mask=hsv_mask).astype(np.uint8)


    
    def process_image_to_df(self, input_image, area_th):
        
        seg_img = self.bg_segmentation(input_image, show_img=False)

    #     # make the mask a binary thresholded image
        mask = cv.cvtColor(seg_img, cv.COLOR_BGR2GRAY)
        mask = cv.GaussianBlur(mask,(5,5),0)
        ret3, mask = cv.threshold(mask,0,255,cv.THRESH_BINARY)



        # output image with contours drawn on the original image
        output_image = input_image.copy()

        # find the contours of the detected objects in the image
        contours, hier = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)


        # create the df that we'll return for this image
        df = pd.DataFrame(columns=['color'])

        # # reset the object num
        object_num = 0
        
        for cnt in contours:
            # draw contours on the output image for our personal enjoyment
            cv.drawContours(output_image, [cnt], 0, color=(255, 255, 255), thickness=5)
            # CALCULATE ALL THE CONTOUR SHAPE FEATURES

            # get the x, y, w, h of the bounding rect for the contour
            x, y, w, h = cv.boundingRect(cnt)

            # contour features
            area = cv.contourArea(cnt)
            rect_area = w * h
            fullosity = area / rect_area

            aspect_ratio = float(w)/h
            extent = float(area/ rect_area)
            hull = cv.convexHull(cnt)
            hull_area = cv.contourArea(hull)
            solidity = float(area)/hull_area

            eq_diameter = np.sqrt(4*area/np.pi)

            M= cv.moments(cnt)
            cx= int(M['m10']/M['m00'])
            cy= int(M['m01']/M['m00'])

            # take this rectangle as a subset of the input_image, and calculate things within it
            img_subset = input_image[y:y+h, x:x+w, :]

            # convert to hsv for extracting those values
            img_subset_hsv = cv.cvtColor(img_subset, cv.COLOR_BGR2HSV)

            # FILTER OUT THE WEIRD ONES
            # get rid of tiny objects that are probably noisef
            if area > area_th:
                # draw a blank canvas to put the contour onto, JUST THIS ONE not the others
                # this is a mask
                cimg_justthiscontour = np.zeros_like(input_image)

                # draw the contours on the blank canvas which is original sized
                cv.drawContours(cimg_justthiscontour, [cnt], 0, color=(255, 255, 255), thickness=-1)

                # now take the subset of just the area around the contour of interest
                cimg_subset = cimg_justthiscontour[y:y+h, x:x+w, :]

                # make a binary mask
                cimg_mask = cv.cvtColor(cimg_subset, cv.COLOR_BGR2GRAY)


                ret2, mask = cv.threshold(cimg_mask,0,255,cv.THRESH_BINARY)

                # draw contours on the output image for our personal enjoyment
                cv.drawContours(output_image, [cnt], 0, color=(255, 255, 255), thickness=5)

                img_subset = cv.bitwise_and(img_subset, img_subset, mask=mask).astype(np.uint8)

                # calculate where the object is
                pts = np.where(cimg_subset == 255)
                hue = img_subset_hsv[pts[0], pts[1], 0]
                sat = img_subset_hsv[pts[0], pts[1], 1]
                val = img_subset_hsv[pts[0], pts[1], 2]
                r = img_subset[pts[0], pts[1], 0]
                g = img_subset[pts[0], pts[1], 1]
                b = img_subset[pts[0], pts[1], 2]

                # and export the image for later analysis with something else like a neural network
                cv.imwrite(f"images/train/XX_{object_num}_{randint(10000,99999)}.png", img_subset)

                # add the object labels to the cimg for identification
                cv.putText(output_image, text= str(object_num), 
                           org=(cx - 5,cy - 5), 
                           fontFace= cv.FONT_HERSHEY_SIMPLEX,
                           fontScale=3, 
                           color=(255,255,255), 
                           thickness=5, 
                           lineType=cv.LINE_AA)

        #         print(r.mean(), g.mean(), b.mean(), gli.mean())
                df = df.append({'color' : 0,
                                'x': x,
                                'y': y,
                                'object_num': object_num,
                                'r': r.mean(),
                                'g': g.mean(),
                                'b': b.mean(),
                                'hue': hue.mean(),
                                'sat': sat.mean(),
                                'val': val.mean()
                                 }, ignore_index=True)

                # last thing we do on this loop is increment the object_num
                object_num += 1

            #

        # end result should be a pandas dataframe and the contour image with numbers
        return df.sort_values(by='object_num', axis=0, ascending=True), output_image
    
    def hsv_slide_tool(self, image):
        
        def empty(a):
            pass
        
        h, w = int(image.shape[1]/2), int(image.shape[0]/2)
        cv.namedWindow('masked_image', cv.WINDOW_NORMAL)
        cv.resizeWindow('masked_image', h, w)
        
        cv.namedWindow("trackbars")
        cv.resizeWindow("trackbars", 800, 300)
                
        # color mask trackbars
        cv.createTrackbar("hue_min", "trackbars", 0, 179, empty)
        cv.createTrackbar('hue_max', 'trackbars', 179, 179, empty)
        cv.createTrackbar('sat_min', 'trackbars', 0, 255, empty)
        cv.createTrackbar('sat_max', 'trackbars', 255, 255, empty)
        cv.createTrackbar('val_min', 'trackbars', 0, 255, empty)
        cv.createTrackbar('val_max', 'trackbars', 255, 255, empty)
       

        while True:
            # get image
            img_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
            
            # get trackbar positions
            h_min = cv.getTrackbarPos("hue_min", "trackbars")
            h_max = cv.getTrackbarPos('hue_max', 'trackbars')
            s_min = cv.getTrackbarPos('sat_min', 'trackbars')
            s_max = cv.getTrackbarPos('sat_max', 'trackbars')
            v_min = cv.getTrackbarPos('val_min', 'trackbars')
            v_max = cv.getTrackbarPos('val_max', 'trackbars')
            
            #         self.black_lower = (0, 0, 0)
#         self.black_upper = (179, 255, 30)
#         self.hsv_lower = (0, 0, 100)
#         self.hsv_upper = (179, 255, 255)
            
            # create mask
            hsv_lower = np.array([h_min, s_min, v_min])
            hsv_upper = np.array([h_max, s_max, v_max])
            black_lower = np.array([0, 0, 0])
            black_upper = np.array([179, 255, 30])
            
            color_mask = cv.inRange(img_hsv, hsv_lower, hsv_upper)
            black_mask = cv.inRange(img_hsv, black_lower, black_upper)
            mask = color_mask + black_mask
            masked_image = cv.bitwise_and(img_hsv, img_hsv, mask=mask)
            
            cv.imshow('masked_image', masked_image)
            k = cv.waitKey(1000) & 0xFF # large wait time
            if k == 113 or k == 27:
                break
        
        cv.destroyAllWindows()
        print(f'hsv_lower is {hsv_lower}, hsv_upper = {hsv_upper}')
              
 
        
    def label_dataframe(self, image_df, class_list):
        for i, row in image_df.iterrows():
            image_df.loc[i, 'color'] = class_list[i]
        print(type(image_df))
        return image_df
    
#     def fake_df(self, input_df, reps = 3):
#         # creates a bunch of fake adjustments to the dataframe so my train set is bigger
#         output_df = input_df.copy()
        
#         for rep in range(0, reps):
#             fake_df = input_df.copy()
#             for i, row in fake_df.iterrows():
#                 fake_df.loc[i, 'r'] = fake_df.loc[i, 'r'] + uniform(-.1, .1)
#                 fake_df.loc[i, 'g'] = fake_df.loc[i, 'g'] + uniform(-.1, .1)
#                 fake_df.loc[i, 'b'] = fake_df.loc[i, 'b'] + uniform(-.1, .1)
#             output_df = pd.concat(output_df, fake_df)
                
#         return output_df
        
    def otsu_threshold(self, image):
        blur = cv.GaussianBlur(image,(5,5),0)
        ret3,th3 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
        return ret3, th3

    def process_image_make_predictions(self, input_image, model):
        
        predictive_model = model
        
        
        area_th = 400
        
        seg_img = self.bg_segmentation(input_image, show_img=False)

    #     # make the mask a binary thresholded image
        mask = cv.cvtColor(seg_img, cv.COLOR_BGR2GRAY)
        mask = cv.GaussianBlur(mask,(5,5),0)
        ret3, mask = cv.threshold(mask,0,255,cv.THRESH_BINARY)

        # output image with contours drawn on the original image
        output_image = input_image.copy()

        # find the contours of the detected objects in the image
        contours, hier = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        # create the df that we'll return for this image
        df = pd.DataFrame(columns=['color'])

        # # reset the object num
        object_num = 0
        
        for cnt in contours:
            # draw contours on the output image for our personal enjoyment
            cv.drawContours(output_image, [cnt], 0, color=(255, 255, 255), thickness=5)
            # CALCULATE ALL THE CONTOUR SHAPE FEATURES

            # get the x, y, w, h of the bounding rect for the contour
            x, y, w, h = cv.boundingRect(cnt)

            # contour features
            area = cv.contourArea(cnt)
            rect_area = w * h
            fullosity = area / rect_area

            aspect_ratio = float(w)/h
            extent = float(area/ rect_area)
            hull = cv.convexHull(cnt)
            hull_area = cv.contourArea(hull)
            solidity = float(area)/hull_area

            eq_diameter = np.sqrt(4*area/np.pi)

            M= cv.moments(cnt)
            cx= int(M['m10']/M['m00'])
            cy= int(M['m01']/M['m00'])

            # take this rectangle as a subset of the input_image, and calculate things within it
            img_subset = input_image[y:y+h, x:x+w, :]

            # convert to hsv for extracting those values
            img_subset_hsv = cv.cvtColor(img_subset, cv.COLOR_BGR2HSV)


            # FILTER OUT THE WEIRD ONES
            # get rid of tiny objects that are probably noisef
            if area > area_th:
                # draw a blank canvas to put the contour onto, JUST THIS ONE not the others
                # this is a mask
                cimg_justthiscontour = np.zeros_like(input_image)

                # draw the contours on the blank canvas which is original sized
                cv.drawContours(cimg_justthiscontour, [cnt], 0, color=(255, 255, 255), thickness=-1)

                # now take the subset of just the area around the contour of interest
                cimg_subset = cimg_justthiscontour[y:y+h, x:x+w, :]

                # make a binary mask
                cimg_mask = cv.cvtColor(cimg_subset, cv.COLOR_BGR2GRAY)


                ret2, mask = cv.threshold(cimg_mask,0,255,cv.THRESH_BINARY)

                # draw contours on the output image for our personal enjoyment
                cv.drawContours(output_image, [cnt], 0, color=(255, 255, 255), thickness=5)

                img_subset = cv.bitwise_and(img_subset, img_subset, mask=mask).astype(np.uint8)

                # calculate where the object is
                pts = np.where(cimg_subset == 255)
                hue = img_subset_hsv[pts[0], pts[1], 0]
                sat = img_subset_hsv[pts[0], pts[1], 1]
                val = img_subset_hsv[pts[0], pts[1], 2]
                r = img_subset[pts[0], pts[1], 0]
                g = img_subset[pts[0], pts[1], 1]
                b = img_subset[pts[0], pts[1], 2]
                
                df = [{'r': (r.mean() / 255),
                      'g': (g.mean() / 255),
                      'b': (b.mean() / 255),
                      'hue': (hue.mean() / 255),
                      'sat': (sat.mean() / 255),
                      'val': (val.mean() / 255)}]
                
                
                
                df = pd.DataFrame.from_dict(df)
                
                pred = predictive_model.get_predictions(df)
                
                class_dict = {0:"medium_blue",
                              1:"black",
                              2:"darK_stone_gray",
                              3:"bright_green",
                              4:"light_green",
                              5:"bright_orange",
                              6:"bright_red",
                              7:"bright_blue",
                              8:"white",
                              9:"bright_yellow"}
                color_text = class_dict[pred[0]]
                
                object_label = "obj" + str(object_num) + "_pred" + str(pred[0])
                print(object_label)    
                
                # add the object labels to the cimg for identification
                cv.putText(output_image, text= str(object_label), 
                           org=(cx - 5,cy - 5), 
                           fontFace= cv.FONT_HERSHEY_SIMPLEX,
                           fontScale=1,
                           color=(0,255,0), 
                           thickness=3,
                           lineType=cv.LINE_AA)
                
                # last thing we do on this loop is increment the object_num
                object_num += 1
        
        # AFTER ALL CONTOURS HAVE BEEN DONE submit the df to the model for predictions
        
#         results = predictive_model.blind_predictions()
        # result = loaded_model.get_predictions(X_test, Y_test)
        # print(result)

        # # use the test set to see how we do
        # y_test_predictions = nb.get_predictions(X_test)

        # # scores
        # acc = nb.get_accuracy(y_test, y_test_predictions)
        # prec = precision_score(y_test, y_test_predictions, average="micro")
        # rec = recall_score(y_test, y_test_predictions, average="micro")
        # print(f"precision is {prec}, recall is {rec}, accuracy = {acc}")

        # # confusion matrix
        # labels = [(i, c) for i, c in class_dict.items()]
        # cm = confusion_matrix(y_test, y_test_predictions)
        # fig = plt.figure()
        # ax = fig.add_subplot(111)
        # cax = ax.matshow(cm)
        # plt.title('confusion matrix of the classifier')
        # fig.colorbar(cax)
        # plt.xlabel('Predicted')
        # plt.ylabel('True')
        # plt.show()
        # print(labels)

            # take the row
            

        
        
        # end result should be a pandas dataframe and the contour image with numbers
        return output_image

Writing pylegoclassifier.py


In [ ]:
matlab = MatlabSurrogate()
imageproc = ImageProcess()


# get our raw images into a list
raw_img_path = "images/test/"
lego_imgs = [(raw_img_path + f) for f in listdir(raw_img_path) if isfile(join(raw_img_path, f))]
print(f"there are {len(lego_imgs)} lego images in list")

idx = 0

lego_img = matlab.acquire_kinect_image(lego_imgs[idx])

nb_model = pickle.load(open('nb_classifier.sav', 'rb'))

result_image = imageproc.process_image_make_predictions(lego_img, nb_model)
matlab.imshow(result_image)


# imageproc = ImageProcess()


# # get our raw images into a list
# from os import listdir
# from os.path import isfile, join
# raw_img_path = "images/raw/"
# lego_imgs = [(raw_img_path + f) for f in listdir(raw_img_path) if isfile(join(raw_img_path, f))]


# idx = 0
# # arguments to pass to the function
# test_img1 = matlab.acquire_kinect_image(lego_imgs[idx])


# # this will be in a function


            

            
        

   

    
        
# df2, output_img = dummy_process(test_img1, area_th = 100)
# matlab.imshow(output_img)


In [ ]:
########### Contructing the training data set #################


# instantiate objects
imageproc = ImageProcess() # does the background segmentation and other image processing methods, also data extraction
matlab = MatlabSurrogate() # does some image loading and display, pretending we're using some 

# get our raw images into a list
raw_img_path = "images/raw/"
lego_imgs = [(raw_img_path + f) for f in listdir(raw_img_path) if isfile(join(raw_img_path, f))]
print(f"there are {len(lego_imgs)} lego images in list")

# ['images/raw/legos_0.png', 
#  'images/raw/legos_1.jpg', 
#  'images/raw/Sample1.jpg', 
#  'images/raw/Sample10.jpg', 
#  'images/raw/Sample11.jpg', 
#  'images/raw/Sample12.jpg', 
#  'images/raw/Sample13.jpg', 
#  'images/raw/Sample14.jpg', 
#  'images/raw/Sample15.jpg', 
#  'images/raw/Sample2.jpg', 
#  'images/raw/Sample3.jpg', 
#  'images/raw/Sample4.jpg', 
#  'images/raw/Sample5.jpg', 
#  'images/raw/Sample6.jpg', 
#  'images/raw/Sample7.jpg', 
#  'images/raw/Sample8.jpg', 
#  'images/raw/Sample9.jpg']

# create a dummy list, we'll replace these with actual class assignments as I go
dummy_list = [x for x in range(0, 1000)]

# define the classes for each images
# img_classes = [[0, 0, 7, 3, 4, 6, 2, 0, 2, 11, 9, 1, 10, 5],
#               [10, 5, 1, 9, 11, 2, 8, 7]]
img_classes = []

# fill the remainded of the list that doesn't have classification already done with the dummy list
# this is a new way to use list comprehension for me
[img_classes.append(dummy_list) for x in range(len(lego_imgs) - len(img_classes))]

train_df = pd.DataFrame()
# f_ = pd.DataFrame(index=index, columns=columns)
# df_ = df_.fillna(0) # with 0s rather than NaNs

# class_dict = {0: "background",
#               1:"bright_yellow",
#               2: "medium_blue",
#               3: "darK_stone_gray",
#               4:"white",
#               5:"bright_red",
#               6:"medium_lilac",
#               7:"black",
#               8:"bright_blue",
#               9:"light_green",
#               10:"bright_orange",
#               11:"bright_green"}

# legos_0 = matlab.acquire_kinect_image(lego_imgs[0])
# legos_1 = matlab.acquire_kinect_image(lego_imgs[1])

for i, img in enumerate(lego_imgs):
    #load
    input_img = matlab.acquire_kinect_image(img)
    
    # normalize
    lego_img = np.zeros(input_img.shape)
    lego_img = cv.normalize(input_img,  lego_img, 0, 255, cv.NORM_MINMAX)
    
#     # segment
#     seg_img = imageproc.bg_segmentation(lego_img, show_img=True)
    
    # load class list
    lego_img_cl = img_classes[i]
    
    # process image to datagrame
    # takes a segmented image and draws contours
    image_df, output_img = imageproc.process_image_to_df(lego_img, area_th = 100)
    
    # label dataframe from class list
    image_df = imageproc.label_dataframe(image_df, class_list = lego_img_cl)

    # get image name from original filename
    image_name = lego_imgs[i].split('/')[2].split('.')[0]

    # write output_image to disk with color codes
    cv.imwrite(f"images/classification_{i}_{image_name}.png", output_img)
    
    # add the dataframe to the training dataframe
    if i==0:
        train_df = image_df
    else:
        train_df = pd.concat([train_df, image_df])

# save the training dataframe to disk
train_df.to_csv("trainingdata.csv")


# class_dict = {0: "background",
#               1:"bright_yellow",
#               2: "medium_blue",
#               3: "darK_stone_gray",
#               4:"white",
#               5:"bright_red",
#               6:"medium_lilac",
#               7:"black",
#               8:"bright_blue",
#               9:"light_green",
#               10:"bright_orange",
#               11:"bright_green"}



In [ ]:
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

# idx = 12
# lego_img = matlab.acquire_kinect_image(lego_imgs[idx])
# print(lego_imgs[idx])
# # imageproc.hsv_slide_tool(lego_img)




# # img = cv.cvtColor(lego_img, cv.COLOR_BGR2GRAY)

# # global thresholding
# ret1,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# # Otsu's thresholding
# ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# # Otsu's thresholding after Gaussian filtering
# blur = cv2.GaussianBlur(img,(5,5),0)
# ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# # plot all the images and their histograms
# images = [img, 0, th1,
#           img, 0, th2,
#           blur, 0, th3]
# titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
#           'Original Noisy Image','Histogram',"Otsu's Thresholding",
#           'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]

# plt.rcParams["figure.figsize"]=20,20

# for i in range(3):
#     plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
#     plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
#     plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
#     plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
#     plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
#     plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
# plt.show()

In [ ]:
# import the labeled dataset and scale it
df = pd.read_csv("trainingdata_labeled.csv")
color_classes = np.unique(df['color_letter'])

for i, color_letter in enumerate(color_classes):
    print(i, color_letter)
    df.loc[df['color_letter'] == color_letter, 'color'] = i

dataset = df.drop(['color_letter', 'x', 'y', 'object_num'], axis=1)

class_dict = {0:"medium_blue",
              1:"black",
              2:"darK_stone_gray",
              3:"bright_green",
              4:"light_green",
              5:"bright_orange",
              6:"bright_red",
              7:"bright_blue",
              8:"white",
              9:"bright_yellow"}

# data and labels
y = dataset['color']
X = dataset.drop(['color'], axis=1)
X = X / 255
print(X)

# scalerX = preprocessing.StandardScaler().fit(X.iloc[:,1:])
# X_scaled = scalerX.fit_transform(X.iloc[:,1:])


print(np.unique(y))


# # split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

print(f'the type for X_train is {type(X_train)}, the type for X is {type(X)}')
# # initialize the Naive Bayes class as an object
nb = NaiveBayes()

# # sumnmarize the dataset to train the model
# # this gets class means, var, priors, etc
nb.fit(X_train, y_train)

# # # # make predictions using the train set
y_train_predictions = nb.get_predictions(X_train)
acc = nb.get_accuracy(y_train, y_train_predictions)
prec = precision_score(y_train, y_train_predictions, average="micro")
rec = recall_score(y_train, y_train_predictions, average="micro")
print(f"precision is {prec}, recall is {rec}, accuracy = {acc}")

# save the model to disk
filename = 'nb_classifier.sav'
pickle.dump(nb, open(filename, 'wb'))
 
# some time later...
 

In [ ]:
# make predictions from a saved model

# load the model from disk
filename = 'nb_classifier.sav'
nb_model = pickle.load(open(filename, 'rb'))



def get_prediction_from_image():
    
    #load
    input_img = matlab.acquire_kinect_image(img)
    
    # normalize
    lego_img = np.zeros(input_img.shape)
    lego_img = cv.normalize(input_img,  lego_img, 0, 255, cv.NORM_MINMAX)
    
#     # segment
#     seg_img = imageproc.bg_segmentation(lego_img, show_img=True)
    
    # load class list
    lego_img_cl = img_classes[i]
    
    # process image to datagrame
    # takes a segmented image and draws contours
    image_df, output_img = imageproc.process_image_to_df(lego_img, area_th = 100)
    
    # label dataframe from class list
    image_df = imageproc.label_dataframe(image_df, class_list = lego_img_cl)

    # get image name from original filename
    image_name = lego_imgs[i].split('/')[2].split('.')[0]

    # write output_image to disk with color codes
    cv.imwrite(f"images/classification_{i}_{image_name}.png", output_img)
    
    # add the dataframe to the training dataframe
    if i==0:
        train_df = image_df
    else:
        train_df = pd.concat([train_df, image_df])


















# result = loaded_model.get_predictions(X_test, Y_test)
# print(result)

# # use the test set to see how we do
# y_test_predictions = nb.get_predictions(X_test)

# # scores
# acc = nb.get_accuracy(y_test, y_test_predictions)
# prec = precision_score(y_test, y_test_predictions, average="micro")
# rec = recall_score(y_test, y_test_predictions, average="micro")
# print(f"precision is {prec}, recall is {rec}, accuracy = {acc}")

# # confusion matrix
# labels = [(i, c) for i, c in class_dict.items()]
# cm = confusion_matrix(y_test, y_test_predictions)
# fig = plt.figure()
# ax = fig.add_subplot(111)
# cax = ax.matshow(cm)
# plt.title('confusion matrix of the classifier')
# fig.colorbar(cax)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.show()
# print(labels)

In [ ]:
# rootdir = 'images/train/'

# for subdir, dirs, files in os.walk(rootdir):
#     print(subdir, files)
    
    
idx = 14
lego_img = matlab.acquire_kinect_image(lego_imgs[idx])
print(lego_imgs[idx])
hsv_lower, hsv_upper = imageproc.hsv_slide_tool(lego_img)


# normalizedImg = np.zeros(lego_img.shape)
# normalizedImg = cv.normalize(lego_img,  normalizedImg, 0, 255, cv.NORM_MINMAX)
# cv.imshow('dst_rt', normalizedImg)
# cv.waitKey(0)
# cv.destroyAllWindows()
    

In [ ]:
# from sklearn.externals import joblib 
  
# # Save the model as a pickle in a file 
# joblib.dump(knn, 'filename.pkl') 
  
# # Load the model from the file 
# knn_from_joblib = joblib.load('filename.pkl')  
  
# # Use the loaded model to make predictions 
# knn_from_joblib.predict(X_test) 


In [ ]:
# import os
# from glob import glob
# f
# # classes
# classes = {0:"background",
#           1:"bright_yellow",
#           2: "medium_blue",
#           3: "darK_stone_gray",
#            4:"white",
#            5:"bright_red",
#            6:"medium_lilac",
#            7:"black",
#            8:"bright_blue",
#            9:"light_green",
#            10:"bright_orange",
#            11:"bright_green"}

# # dataset import
# train_dir = 'images/train/'
# train_dir_list = [(train_dir + v + '/') for k, v in classes.items()]

# # test_list = [print(i, jpg) for i, (k, v) in enumerate(classes.items())]

# for i, train_dir in enumerate(train_dir_list):
#     result = list(Path(".").rglob("*.[pP][nN][gG]"))
#     for img in result:
#         train_data.append((img, i))

    
# print(train_data)
# # # Get the list of all the images
# # Ektachrome_Images = Ektachrome_dir.glob('*.jpeg')
# # HP5_Images = HP5_dir.glob('*.jpeg')
# # Lomochrome_Purple_Images = Lomochrome_Purple_dir.glob('*.jpeg')
# # Tri_X_Images = Tri_X_dir.glob('*.jpeg')
# # Velvia_50_Images = Velvia_50_dir.glob('*.jpeg')

# # # An empty list. We will insert the data into this list in (img_path, label) format
# # train_data = []

# # for img in Ektachrome_Images:
# #     train_data.append((img,1))

# # for img in HP5_Images:
# #     train_data.append((img, 2))

# # for img in Lomochrome_Purple_Images:
# #     train_data.append((img, 3))

# # for img in Tri_X_Images:
# #     train_data.append((img, 4))

# # for img in Velvia_50_Images:
# #     train_data.append((img, 5))








# # # test_image = matlab.acquire_kinect_image("images/legos_0.png")

# # # # use the segmentation function to segment the image.
# # # seg_image = imageproc.bg_segmentation(test_image)

# # # matlab.imshow(seg_image)

In [ ]:
# df.head()

# hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

# # create an hsv mask
# test_image = cv.inRange(hsv_image, 
#                          (50, 20, 0),
#                          (160, 255, 255)).astype(np.uint8)

# test_image = cv.bitwise_and(image, image, mask =test_image).astype(np.uint8)
# print(test_image[0])

# plt.imshow(test_image)

In [ ]:
# # import the cherry images
# # C:\data\BSYSE_530\machine_vision\images\Cherries
# # there are five, with different light conditions
# # DSC_0052, 0054, 0056, 0057, 0058
# # we need to take these images and cut them into little pieces for the process to work

# # convert them to RGB
# images = [cv.cvtColor(cv.imread("C:/data/BSYSE_530/machine_vision/images/Cherries/DSC_0052.jpg"), cv.COLOR_BGR2RGB),
#           cv.cvtColor(cv.imread("C:/data/BSYSE_530/machine_vision/images/Cherries/DSC_0054.jpg"), cv.COLOR_BGR2RGB),
#           cv.cvtColor(cv.imread("C:/data/BSYSE_530/machine_vision/images/Cherries/DSC_0056.jpg"), cv.COLOR_BGR2RGB),
#           cv.cvtColor(cv.imread("C:/data/BSYSE_530/machine_vision/images/Cherries/DSC_0057.jpg"), cv.COLOR_BGR2RGB),
#           cv.cvtColor(cv.imread("C:/data/BSYSE_530/machine_vision/images/Cherries/DSC_0058.jpg"), cv.COLOR_BGR2RGB)]

# titles = ["DSC_0052", "DSC_0054", "DSC_0056","DSC_0057","DSC_0058"]

# masked_images = []
# masks = []
# adj_images = []

# # # # image adjustment, rescale intensity
# # for i in range(0, 5):
# #     img = images[i]
# #     p2, p98 = np.percentile(img, (2, 98))
# #     adj_img = exposure.rescale_intensity(img, in_range=(p2, p98))
# #     adj_images.append(adj_img)
    
# # create the mask
# # try to screen out all the white regions
# background_mask = cv.inRange(images[0],
#                              (70,70,90),
#                              (120,120,120)).astype(np.int) * -1
# print(background_mask.shape)
# print(type(background_mask))
# # background_mask = morphology.binary_dilation(background_mask, np.ones((3, 3)))
# # closing
# background_mask = morphology.closing(background_mask, morphology.disk(2))

# # print(background_mask.shape)
# # print(background_mask)
# # print(np.mean(images[0][650:700,400:500,0]), np.mean(images[0][600:700,0:100,1]), np.mean(images[0][600:700,0:100,2]))

# # now use BGR2HSV to reverse the red and blue to make it easier for hsv filtering of the red (not around 0/360 break)
# hsv_image = cv.cvtColor(images[0], cv.COLOR_BGR2HSV)

# # create an hsv mask
# cherry_mask = cv.inRange(hsv_image, 
#                          (70, 30, 20),
#                          (255, 255, 255)).astype(np.int)


# cherry_mask = get_tgi_mask(cv.cvtColor(cv.imread("C:/data/BSYSE_530/machine_vision/images/Cherries/DSC_0056.jpg"), cv.COLOR_BGR2RGB).astype(np.float64))

# # make that array of truth values 0 or 255 into a 1 0 array
# # cherry_mask = np.where(cherry_mask > 250, 1, 0).astype(np.int)

# # median filter to denoise
# # cherry_mask = ndimage.median_filter(cherry_mask, size=(3, 3)).astype(np.int)


# # do a little dilation to make the mask look nice
# cherry_mask = morphology.binary_dilation(cherry_mask, np.ones((3, 3)))

# # closing
# # cherry_mask = morphology.closing(cherry_mask, morphology.disk(4))

# # erode the mask
# cherry_mask = morphology.erosion(cherry_mask, morphology.disk(2))

# #combine the cherry mask and the background mask
# # cherry_mask = cherry_mask + background_mask

# for image in images:

#     # apply the mask
#     masked_image = np.zeros(image.shape)
#     for channel in range(image.shape[2]):
#         masked_image[:,:,channel] = image[:,:,channel] * cherry_mask
    
#     # the images are going back into "BGR" but thats really RGB
#     masked_images.append(masked_image.astype(np.uint8))

# # # show the images from the last batch just for kicks
# # plot_images(titles=["cherry_mask"], 
# #             images=[cherry_mask],
# #             fsize=30)



# # # show the images from the last batch just for kicks
# plot_images(titles=titles, 
#             images=masked_images,
#             fsize=30)

In [ ]:
# df = pd.DataFrame(columns=['y'])

# # produce the individual images we are going to use for our data set in the neural network step
# for light_level, img_rgb in enumerate(masked_images):

#     # create the image subsets and name them as appropriate for location
#     cherry_0_0 = img_rgb[100:200,200:300,:]
#     cherry_0_1 = img_rgb[80:180,300:400,:]
#     cherry_0_2 = img_rgb[90:190,375:475,:]
#     cherry_0_3 = img_rgb[100:200,500:600,:]
#     cherry_0_4 = img_rgb[100:200,600:700,:]
#     cherry_0_5 = img_rgb[100:200,700:800,:]

#     cherry_1_0 = img_rgb[225:325,190:290,:]
#     cherry_1_1 = img_rgb[225:325,275:375,:]
#     cherry_1_2 = img_rgb[225:325,375:475,:]
#     cherry_1_3 = img_rgb[225:325,500:600,:]
#     cherry_1_4 = img_rgb[225:325,600:700,:]
#     cherry_1_5 = img_rgb[225:325,700:800,:]

#     cherry_2_0 = img_rgb[375:475,175:275,:]
#     cherry_2_1 = img_rgb[375:475,275:375,:]
#     cherry_2_2 = img_rgb[375:475,375:475,:]
#     cherry_2_3 = img_rgb[375:475,500:600,:]
#     cherry_2_4 = img_rgb[375:475,600:700,:]
#     cherry_2_5 = img_rgb[375:475,700:800,:]
    
#     rectangle_0 = img_rgb[525:550,350:350 + 25,:]
#     rectangle_1 = img_rgb[525:550,382:382 + 25,:]
#     rectangle_2 = img_rgb[527:552,415:415 + 25,:]
#     rectangle_3 = img_rgb[527:552,450:450 + 25,:]
#     rectangle_4 = img_rgb[528:553,484:484 + 25,:]
#     rectangle_5 = img_rgb[528:553,519:519 + 25,:]
#     rectangle_6 = img_rgb[529:554,554:554 + 25,:]
        
#     sticky_note = img_rgb[250:430,800:1000,:]

#     images = [cherry_0_0, cherry_0_1, cherry_0_2, cherry_0_3, cherry_0_4, cherry_0_5,
#               cherry_1_0, cherry_1_1, cherry_1_2, cherry_1_3, cherry_1_4, cherry_1_5,
#               cherry_2_0, cherry_2_1, cherry_2_2, cherry_2_3, cherry_2_4, cherry_2_5,
#               rectangle_0, rectangle_1, rectangle_2, rectangle_3, rectangle_4, rectangle_5,
#               rectangle_6, sticky_note]

# #     labels = ["light_color_cherry", "light_color_cherry", "light_color_cherry", "light_color_cherry", "light_color_cherry", "light_color_cherry",
# #               "moderate_color_cherry", "moderate_color_cherry", "moderate_color_cherry", "moderate_color_cherry", "moderate_color_cherry", "moderate_color_cherry",
# #               "dark_color_cherry", "dark_color_cherry", "dark_color_cherry", "dark_color_cherry", "dark_color_cherry", "dark_color_cherry",
# #               "light_color_rectangle", "light_color_rectangle", "moderate_color_rectangle", "moderate_color_rectangle", "moderate_color_rectangle", "dark_color_rectangle",
# #               "dark_color_rectangle", "sticky_notes"]

#     labels = [0, 0, 0, 0, 0, 0,
#               1, 1, 1, 1, 1, 1,
#               2, 2, 2, 2, 2, 2,
#               3, 3, 4, 4, 4, 5, 5, 6]
    
#     labels_dict = {0: "light_color_cherries",
#                   1: "moderate_color_cherries",
#                   2: "dark_color_cherries",
#                   3: "light_color_rectangles",
#                   4: "moderate_color_rectangles",
#                   5: "dark_color_rectangles",
#                   6: "sticky_notes"}
    
#     titles = ["cherry_0_0", "cherry_0_1", "cherry_0_2", "cherry_0_3", "cherry_0_4", "cherry_0_5",
#               "cherry_1_0", "cherry_1_1", "cherry_1_2", "cherry_1_3", "cherry_1_4", "cherry_1_5",
#               "cherry_2_0", "cherry_2_1", "cherry_2_2", "cherry_2_3", "cherry_2_4", "cherry_2_5",
#               "rectangle_0", "rectangle_1", "rectangle_2", "rectangle_3", "rectangle_4", "rectangle_5",
#               "rectangle_6", "sticky_note"]

    
#     # iterate through the zone of interest images
#     for i, image in enumerate(images):
                
# #         # set file name with light level and image title                       
# #         filename =  str(labels[i]) + " " + titles[i] + "_" + str(light_level) + ".jpg"
               
# #         # resize all images to same size for later use
# #         bgr_image = cv.resize(image, (100,100), interpolation = cv.INTER_AREA)
# #         bgr_image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
# #         cv.imwrite("cherries/" + filename, bgr_image)    

# #         # do your dataset creation right here. 
# #         hsv_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2HSV)
        
#         # 
#         p1, p2 = np.percentile(image[:,:,0], (2, 99))
#         red_channel = exposure.rescale_intensity(image[:,:,0], in_range=(p1, p2))
#         blue_channel = exposure.rescale_intensity(image[:,:,1], in_range=(p1, p2))
#         green_channel = exposure.rescale_intensity(image[:,:,2], in_range=(p1, p2))
            
#         test_image = image.astype(np.float64)
#         r = test_image[:,:,0] / np.max(test_image[:,:,0])
#         g = test_image[:,:,1] / np.max(test_image[:,:,1])
#         b = test_image[:,:,2] / np.max(test_image[:,:,2])
        
#         #  gli, ngrdi, r_bg, rbg, tgi*, br, rg
#         rg_index_labels = ["gli", "ngrdi", "r_bg", "rbg", "tgi", "br", "rg"]
#         rg_index = [calc_index(test_image, idx) for idx in rg_index_labels]

#         # get the binary mask for this image, convert to unsigned 8-bit int
#         bin_image = get_tgi_mask(image)
#         print(type(bin_image), bin_image.dtype)
#         contours, hier = cv.findContours(bin_image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#         cnt = contours[0]
#         x, y, w, h = cv.boundingRect(cnt)
        
#         area = np.sum(bin_image)
#         cnt_area = cv.contourArea(cnt)
#         aspect_ratio = float(w)/h
#         rect_area = w * h
#         extent = float(cnt_area)/rect_area
#         hull = cv.convexHull(cnt)
#         hull_area = cv.contourArea(hull)
#         solidity = float(cnt_area)/hull_area
#         eq_diameter = np.sqrt(4*cnt_area/np.pi)
    
    
        
#         # try converting the images to pandas data frames, each of these channels and indices as a reshaped column. 
#         # then use pandas data frame commands to get some values
#         df_images = pd.DataFrame()
#         df_images["r_rs"] = np.ndarray.flatten(red_channel)
#         df_images["b_rs"] = np.ndarray.flatten(green_channel)
#         df_images["g_rs"] = np.ndarray.flatten(blue_channel)
#         df_images["r"] = np.ndarray.flatten(r)
#         df_images["b"] = np.ndarray.flatten(g)
#         df_images["g"] = np.ndarray.flatten(b)
#         df_images["gli"] = np.ndarray.flatten(rg_index[0])
#         df_images["ngrdi"] = np.ndarray.flatten(rg_index[1])
#         df_images["r_bg"] = np.ndarray.flatten(rg_index[2])
#         df_images["rbg"] = np.ndarray.flatten(rg_index[3])
#         df_images["tgi"] = np.ndarray.flatten(rg_index[4])
#         df_images["br"] = np.ndarray.flatten(rg_index[5])
#         df_images["rg"] = np.ndarray.flatten(rg_index[6])
               
#         df = df.append({'y' : labels[i],
#                         'mean_r_rs': df_images.r_rs[df_images.r_rs > 0].mean(),
#                         'mean_g_rs': df_images.g_rs[df_images.g_rs > 0].mean(),
#                         'mean_b_rs': df_images.b_rs[df_images.b_rs > 0].mean(),
#                         'area': area,
#                         "cnt_area": cnt_area,
# #                         "aspect_ratio": aspect_ratio,
# #                         "rect_area": rect_area,
# #                         "extent": extent,
# #                         "hull_area": hull_area, 
# #                         "solidity": solidity,
# #                         "eq_diameter": eq_diameter,
#                         'mean_r': df_images.r[df_images.r > 0].mean(),
#                         'mean_g': df_images.g[df_images.g > 0].mean(),
#                         'mean_b': df_images.b[df_images.b > 0].mean(),
#                         'gli': df_images.gli[df_images.gli < 0].mean(),
# #                         'ngrdi': df_images.ngrdi[df_images.ngrdi < 0].mean(),
#                         'r_bg': df_images.r_bg.mean(),
#                         'rbg': df_images.rbg.mean(),
#                         'tgi': df_images.tgi[df_images.tgi < 0].mean(),
#                         'br': df_images.br[df_images.br < 0].mean(),
#                         'rg': df_images.rg.mean()
#                        }, ignore_index=True)
        

#         # show the images from the last batch just for kicks
# plot_images(titles=rg_index_labels, 
#             images=rg_index,
#             fsize=30)

# for image in rg_index:
#     flat_img = np.ndarray.flatten(image)
#     print(flat_img.min(), flat_img.max())
# print(df)

    

In [ ]:
# # do a wacky thing here
# # wacky_images = [exposure.equalize_hist(img[:,:,0]) for img in images]
# # wacky_images = [exposure.equalize_adapthist(img[:,:,0]) for img in images]

# test_image = cv.cvtColor(cv.imread("C:/data/BSYSE_530/machine_vision/images/Cherries/DSC_0052.jpg"), cv.COLOR_BGR2RGB).astype(np.float64)
# r = test_image[:,:,0] / np.max(test_image[:,:,0])
# g = test_image[:,:,1] / np.max(test_image[:,:,1])
# b = test_image[:,:,2] / np.max(test_image[:,:,2])


# #  gli, ngrdi, r_bg, rbg, tgi*, br, rg
# rg_index_labels = ["gli", "ngrdi", "r_bg", "rbg", "tgi", "br", "rg"]
# rg_index = [calc_index(test_image, idx) for idx in rg_index_labels]

# # show the images from the last batch just for kicks
# plot_images(titles=rg_index_labels, 
#             images=rg_index,
#             fsize=15)
